In [ ]:
import pandas as pd
import pickle
import numpy as np

# 데이터 불러오기
final_df = pd.read_csv("/content/drive/MyDrive/dataset/final(0519).csv")

In [ ]:
# 결측치 처리
final_df["Title"].fillna("제목 미입력", inplace=True)
final_df["LocationDescription"].fillna("위치 정보 없음", inplace=True)
final_df["MainDescription"].fillna("상세 설명 없음", inplace=True)
final_df["SpaceDescription"].fillna("공간 설명 없음", inplace=True)
final_df["GuestAccessDescription"].fillna("이용 가능 공간 정보 없음", inplace=True)
final_df["Host_ResponseRate"].fillna("응답률 정보 없음", inplace=True)
final_df["Host_ResponseTime"].fillna("응답 시간 정보 없음", inplace=True)
final_df["amenities_SelfCheckIn"].fillna("셀프 체크인 여부 모름", inplace=True)
final_df["CheckIn_Start"].fillna("체크인 시간 모름", inplace=True)
final_df["CheckOut_End"].fillna("체크아웃 시간 모름", inplace=True)

# 매핑 딕셔너리 정의
neighborhood_mapping = {
    "Ma-po-gu": "마포구",
    "Yong-san-gu": "용산구",
    "Yongsan-gu": "용산구",
    "Gang-nam-gu": "강남구",
    "Jong-no-gu": "종로구",
    "Jung-gu": "중구",
    "Myeong-dong": "명동",
    "Gwa-nak-gu": "관악구",
    "Yeong-deung-po-gu": "영등포구",
    "Gwang-jin-gu": "광진구",
    "Seo-cho-gu": "서초구",
    "Seocho-gu": "서초구",
    "Dong-dae-mun-gu": "동대문구",
    "Song-pa-gu": "송파구",
    "Seo-dae-mun-gu": "서대문구",
    "Seong-buk-gu": "성북구",
    "Eun-pyeong-gu": "은평구",
    "Seong-dong-gu": "성동구",
    "Dong-jak-gu": "동작구",
    "Gang-buk-gu": "강북구",
    "Sin-chon-dong": "신촌동",
    "Jung-nang-gu": "중랑구",
    "Seong-su 2 ga 1 dong": "성수2가1동",
    "Gu-ro-gu": "구로구",
    "Geum-cheon-gu": "금천구",
    "No-won-gu": "노원구",
    "Do-bong-gu": "도봉구",
    "Goyang-si": "고양시"
}

def map_neighborhood_to_korean(name: str) -> str:
    return neighborhood_mapping.get(name, name)  # 없는 값은 그대로 반환

# 새 컬럼 생성
final_df["LocalizedNeighbourhood_ML"] = final_df["LocalizedNeighbourhood_ML"].apply(map_neighborhood_to_korean)

<ipython-input-3-663e9ba00cf7>:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  final_df["Title"].fillna("제목 미입력", inplace=True)
<ipython-input-3-663e9ba00cf7>:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 

In [ ]:
# Calendar_Date를 datetime으로 변환하고 month 컬럼 추가
final_df['Calendar_Date'] = pd.to_datetime(final_df['Calendar_Date'])
final_df['month'] = final_df['Calendar_Date'].dt.month

def compress_days_only(sub_df):
    """Airbnb_ListingID별 월별 예약 가능일 요약"""
    available_dates = sub_df[sub_df['isAvailable']]['Calendar_Date'].sort_values()

    if available_dates.empty:
        return ""

    days = available_dates.dt.day.tolist()
    days.sort()

    compressed = []
    start = prev = days[0]
    for d in days[1:]:
        if d == prev + 1:
            prev = d
        else:
            compressed.append(f"{start}-{prev}" if start != prev else f"{start}")
            start = prev = d
    compressed.append(f"{start}-{prev}" if start != prev else f"{start}")

    return ", ".join(map(str, compressed))

# Airbnb_ListingID, month 단위로 그룹화
available_ranges_df = (
    final_df
    .groupby(['Airbnb_ListingID', 'month'])
    .apply(compress_days_only)
    .reset_index()
)

available_ranges_df.columns = ['Airbnb_ListingID', 'month', 'Available_Ranges']

# 월별 예약 가능 요약이 포함된 price_df로 merge
price_df = final_df.merge(available_ranges_df, on=['Airbnb_ListingID', 'month'], how='left')

<ipython-input-7-f6e4583ab772>:31: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(compress_days_only)


In [ ]:
final_df['dayofweek'] = final_df['Calendar_Date'].dt.dayofweek
price_df = price_df[['Airbnb_ListingID', 'month', 'dayofweek', 'Guests', 'Price_Per_Night', 'Cleaning_Fee', 'Airbnb_Service_Fee', 'total_price', 'Available_Ranges_y']]

In [ ]:
price_df

,Airbnb_ListingID,month,dayofweek,Guests,Price_Per_Night,Cleaning_Fee,Airbnb_Service_Fee,total_price,Available_Ranges_y
0,1000084126553197219,1,0,2.0,80240.0,23600.0,0.0,103840.0,"6-9, 13-16, 20-23"
1,1000084126553197219,1,1,2.0,80240.0,23600.0,0.0,103840.0,"6-9, 13-16, 20-23"
2,1000084126553197219,1,2,2.0,80240.0,23600.0,0.0,103840.0,"6-9, 13-16, 20-23"
3,1000084126553197219,1,3,2.0,80240.0,23600.0,0.0,103840.0,"6-9, 13-16, 20-23"
4,1000084126553197219,1,0,2.0,80240.0,23600.0,0.0,103840.0,"6-9, 13-16, 20-23"
...,...,...,...,...,...,...,...,...,...
1922840,999533248122822217,12,4,2.0,295000.0,0.0,0.0,295000.0,"1-6, 8-13, 15-16, 16-17, 17-18, 18-19, 19-20, ..."
1922841,999533248122822217,12,5,2.0,295000.0,0.0,0.0,295000.0,"1-6, 8-13, 15-16, 16-17, 17-18, 18-19, 19-20, ..."
1922842,999533248122822217,12,0,2.0,310000.0,0.0,0.0,310000.0,"1-6, 8-13, 15-16, 16-17, 17-18, 18-19, 19-20, ..."
1922843,999533248122822217,12,1,2.0,310000.0,0.0,0.0,310000.0,"1-6, 8-13, 15-16, 16-17, 17-18, 18-19, 19-20, ..."


In [ ]:
price_mean = price_df[['Airbnb_ListingID', 'month', 'dayofweek', 'Guests', 'Price_Per_Night', 'Cleaning_Fee', 'Airbnb_Service_Fee', 'total_price']].groupby(['Airbnb_ListingID', 'month', 'dayofweek']).mean().reset_index().applymap(lambda x: round(x, 0))
price_max = price_mean[['Airbnb_ListingID', 'total_price']].groupby('Airbnb_ListingID').mean().reset_index().applymap(lambda x: round(x, 0))
price_max.columns = ['Airbnb_ListingID', 'total_price_mean']
price_range = price_df[['Airbnb_ListingID', 'month', 'Available_Ranges_y']]
price_range.columns = ['Airbnb_ListingID', 'month', 'Available_Ranges']
price_range = price_range.drop_duplicates(['Airbnb_ListingID', 'month'])

price_mean = price_mean.merge(price_range, on=['Airbnb_ListingID', 'month'], how='left').merge(price_max, on = ['Airbnb_ListingID'], how = 'left')

<ipython-input-10-34d4e7d210c1>:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  price_mean = price_df[['Airbnb_ListingID', 'month', 'dayofweek', 'Guests', 'Price_Per_Night', 'Cleaning_Fee', 'Airbnb_Service_Fee', 'total_price']].groupby(['Airbnb_ListingID', 'month', 'dayofweek']).mean().reset_index().applymap(lambda x: round(x, 0))
<ipython-input-10-34d4e7d210c1>:2: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  price_max = price_mean[['Airbnb_ListingID', 'total_price']].groupby('Airbnb_ListingID').mean().reset_index().applymap(lambda x: round(x, 0))


In [ ]:
price_mean

,Airbnb_ListingID,month,dayofweek,Guests,Price_Per_Night,Cleaning_Fee,Airbnb_Service_Fee,total_price,Available_Ranges,total_price_mean
0,69022,1,4,4.0,558277.0,138358.0,246967.0,943601.0,"31, 31, 31",1038608.0
1,69022,2,0,4.0,514585.0,218460.0,448074.0,1181119.0,"1, 1, 1, 3, 3, 3-4, 4, 4-5, 5, 5-6, 6, 6-7, 7,...",1038608.0
2,69022,2,1,4.0,514585.0,218460.0,448074.0,1181119.0,"1, 1, 1, 3, 3, 3-4, 4, 4-5, 5, 5-6, 6, 6-7, 7,...",1038608.0
3,69022,2,2,4.0,514585.0,218460.0,448074.0,1181119.0,"1, 1, 1, 3, 3, 3-4, 4, 4-5, 5, 5-6, 6, 6-7, 7,...",1038608.0
4,69022,2,3,4.0,514585.0,218460.0,448074.0,1181119.0,"1, 1, 1, 3, 3, 3-4, 4, 4-5, 5, 5-6, 6, 6-7, 7,...",1038608.0
...,...,...,...,...,...,...,...,...,...,...
311245,1305273861684182884,12,1,4.0,403542.0,80000.0,287013.0,770554.0,"1, 1, 1-2, 2, 2-3, 3, 3-4, 4, 4-5, 5, 5-6, 6, ...",702711.0
311246,1305273861684182884,12,2,4.0,403542.0,80000.0,287013.0,770554.0,"1, 1, 1-2, 2, 2-3, 3, 3-4, 4, 4-5, 5, 5-6, 6, ...",702711.0
311247,1305273861684182884,12,3,4.0,403542.0,80000.0,287013.0,770554.0,"1, 1, 1-2, 2, 2-3, 3, 3-4, 4, 4-5, 5, 5-6, 6, ...",702711.0
311248,1305273861684182884,12,4,4.0,494167.0,80000.0,180989.0,755156.0,"1, 1, 1-2, 2, 2-3, 3, 3-4, 4, 4-5, 5, 5-6, 6, ...",702711.0


In [ ]:
price_mean.head(2)

,Airbnb_ListingID,month,Guests,Price_Per_Night,Cleaning_Fee,Airbnb_Service_Fee,total_price,Available_Ranges,total_price_mean
0,69022,1,4.0,558277.0,138358.0,246967.0,943601.0,"31, 31, 31",983187.0
1,69022,2,4.0,529149.0,191759.0,381038.0,1101946.0,"1, 1, 1, 3, 3, 3-4, 4, 4-5, 5, 5-6, 6, 6-7, 7,...",983187.0


In [ ]:
price_mean

,Airbnb_ListingID,month,Guests,Price_Per_Night,Cleaning_Fee,Airbnb_Service_Fee,total_price,Available_Ranges,total_price_mean,embedding_text
0,69022,1,4.0,558277.0,138358.0,246967.0,943601.0,"31, 31, 31",983187.0,"리스팅ID: 69022\n예약 월: 1의 예약가능 날짜 : 31, 31, 31\n예..."
1,69022,2,4.0,529149.0,191759.0,381038.0,1101946.0,"1, 1, 1, 3, 3, 3-4, 4, 4-5, 5, 5-6, 6, 6-7, 7,...",983187.0,"리스팅ID: 69022\n예약 월: 2의 예약가능 날짜 : 1, 1, 1, 3, 3..."
2,69022,3,4.0,558277.0,138358.0,246967.0,943601.0,"1, 1, 1",983187.0,"리스팅ID: 69022\n예약 월: 3의 예약가능 날짜 : 1, 1, 1\n예약 인..."
3,69022,4,4.0,558277.0,138358.0,246967.0,943601.0,"4, 4, 4-5, 5, 5",983187.0,"리스팅ID: 69022\n예약 월: 4의 예약가능 날짜 : 4, 4, 4-5, 5,..."
4,118591,1,4.0,761126.0,143862.0,0.0,904988.0,"1, 1, 1-2, 2, 2, 6, 6, 6-7, 7, 7-8, 8, 8-9, 9,...",810887.0,"리스팅ID: 118591\n예약 월: 1의 예약가능 날짜 : 1, 1, 1-2, 2..."
...,...,...,...,...,...,...,...,...,...,...
59099,1305273861684182884,8,4.0,395128.0,80000.0,221800.0,696928.0,"1, 1, 1-2, 2, 2, 4, 4, 4-5, 5, 5-6, 6, 6-7, 7,...",702358.0,리스팅ID: 1305273861684182884\n예약 월: 8의 예약가능 날짜 :...
59100,1305273861684182884,9,4.0,386667.0,80000.0,229158.0,695825.0,"1, 1, 1-2, 2, 2-3, 3, 3-4, 4, 4-5, 5, 5-6, 6, ...",702358.0,리스팅ID: 1305273861684182884\n예약 월: 9의 예약가능 날짜 :...
59101,1305273861684182884,10,4.0,401111.0,80000.0,216597.0,697708.0,"1, 1, 1-2, 2, 2-3, 3, 3-4, 4, 4, 10, 10, 10-11...",702358.0,리스팅ID: 1305273861684182884\n예약 월: 10의 예약가능 날짜 ...
59102,1305273861684182884,11,4.0,390000.0,80000.0,226259.0,696259.0,"10, 10, 10-11, 11, 11-12, 12, 12-13, 13, 13-14...",702358.0,리스팅ID: 1305273861684182884\n예약 월: 11의 예약가능 날짜 ...


In [ ]:
price_mean["price_comment"] = price_mean.apply(
    lambda row: (
        "평소 수준의 요금입니다."
        if abs((row["total_price"] - row["total_price_mean"]) / row["total_price_mean"] * 100) <= 5 else
        "평소보다 저렴한 편입니다."
        if (row["total_price"] - row["total_price_mean"]) / row["total_price_mean"] * 100 < -5 else
        "평소보다 다소 높은 편입니다."
    ) + (
        " 특가 숙소로 분류됩니다.\n"
        if (row["total_price"] - row["total_price_mean"]) / row["total_price_mean"] * 100 < -20 else ""
    ),
    axis=1
)

In [ ]:
# ✅ 1. CSV -> Pickle 변환 + FAISS 준비용 벡터 저장 스크립트
import pandas as pd
import pickle
from sentence_transformers import SentenceTransformer
import numpy as np

embedding_model = SentenceTransformer("jhgan/ko-sroberta-multitask")

# 임베딩 텍스트 구성 함수
def build_embedding_text(row):
    # 가격 차이 및 변화율 계산
    price_diff = row['total_price'] - row['total_price_mean']
    price_change_ratio = (price_diff / row['total_price_mean']) * 100 if row['total_price_mean'] else 0

    # 요금 비교 문구 ('평소' 기준 ±5% 허용)
    if abs(price_change_ratio) <= 5:
        price_comment = "평소 수준의 요금입니다."
    elif price_change_ratio < -5:
        price_comment = "평소보다 저렴한 편입니다."
    else:
        price_comment = "평소보다 다소 높은 편입니다."

    # 특가 여부 태그
    special_offer_note = " 특가 숙소로 분류됩니다.\n" if price_change_ratio < -20 else ""

    return (
        f"리스팅ID: {row['Airbnb_ListingID']}\n"
        f"{row['month']}월의 예약가능 날짜 : {row['Available_Ranges']}\n"
        f"예약 인원: {row['Guests']}명\n"
        f"1박 요금: {row['Price_Per_Night']}원\n"
        f"청소비: {row['Cleaning_Fee']}원 / 에어비앤비 수수료: {row['Airbnb_Service_Fee']}원\n"
        f"1박 총요금: {row['total_price']}원\n"
        f"요금 비교: {price_comment} (평소 대비 {price_change_ratio:.1f}% {'상승' if price_change_ratio > 0 else '하락' if price_change_ratio < 0 else '변동 없음'})\n"
        f"{special_offer_note}"
    )

# 임베딩 텍스트와 벡터 생성
price_mean['embedding_text'] = price_mean.apply(build_embedding_text, axis=1)

embeddings = embedding_model.encode(price_mean['embedding_text'].tolist(), show_progress_bar=True, batch_size=64)

# numpy array로 변환
dense_vectors = np.array(embeddings)

# 저장 (Listing ID 포함 DF 유지)
with open("/content/drive/MyDrive/dataset/airbnb_data_price_요일추가.pkl", "wb") as f:
    pickle.dump({
        "df": price_mean[['Airbnb_ListingID', 'embedding_text'] +
                      [c for c in price_mean.columns if c not in ['Airbnb_ListingID', 'embedding_text']]],
        "vectors": dense_vectors
    }, f)

print("✅ Pickle 저장 완료: /content/drive/MyDrive/dataset/airbnb_data_price_요일추가.pkl")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.86k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/495k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/156 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/4864 [00:00<?, ?it/s]

✅ Pickle 저장 완료: /content/drive/MyDrive/dataset/airbnb_data_price_요일추가.pkl
